In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_excel("KTM New Data.xlsx")

In [3]:
df.head()

,ID,Age,Gender,Occupation,Phone Type,Current Bike,Relationship,Response
0,1,53,Male,Professional,Average,180 to 220,Complicated,Not purchased
1,2,27,Female,Self Employed,Low End,No Bike,Single,Purchased
2,3,39,Female,Unemployed,Average,180 to 220,Married,Not purchased
3,4,20,Female,Unemployed,High End,No Bike,Married,Not purchased
4,5,29,Male,Student,Average,180 to 220,Complicated,Purchased


In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
le= LabelEncoder()

In [6]:
df['Gender']=le.fit_transform(df['Gender'])
df['Occupation']=le.fit_transform(df['Occupation'])
df['Phone Type']=le.fit_transform(df['Phone Type'])
df['Current Bike']=le.fit_transform(df['Current Bike'])
df['Relationship']=le.fit_transform(df['Relationship'])
df['Response']=le.fit_transform(df['Response'])

In [7]:
df.head()

,ID,Age,Gender,Occupation,Phone Type,Current Bike,Relationship,Response
0,1,53,1,0,0,1,1,0
1,2,27,0,1,2,4,3,1
2,3,39,0,3,0,1,2,0
3,4,20,0,3,1,4,2,0
4,5,29,1,2,0,1,1,1


In [8]:
df=df.drop(columns=['ID'])


In [9]:
df.head()

,Age,Gender,Occupation,Phone Type,Current Bike,Relationship,Response
0,53,1,0,0,1,1,0
1,27,0,1,2,4,3,1
2,39,0,3,0,1,2,0
3,20,0,3,1,4,2,0
4,29,1,2,0,1,1,1


In [10]:
x=df.drop('Response',axis=1)
y=df[['Response']]

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [12]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=50)

In [13]:
# Logistic Regression & its cross validation
lr=LogisticRegression()
lr.fit(x_train,y_train)
lr_pred=lr.predict(x_test)

In [14]:
accuracy_score(y_test,lr_pred)

0.7631578947368421

In [15]:
#Cross validation
from sklearn.model_selection import cross_val_score 
score_lr=cross_val_score(lr,x,y,cv=10)

In [16]:
score_lr

array([0.53947368, 0.59868421, 0.60526316, 0.66447368, 0.82236842,
       0.86184211, 0.85526316, 0.83552632, 0.82781457, 0.87417219])

In [17]:
score_lr.mean()

0.7484881491808993

In [18]:
#Decision Tree & its cross validation
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(x_train,y_train)
dt_pred=dt.predict(x_test)
accuracy_score(y_test,dt_pred)

0.7741228070175439

In [19]:
score_dt=cross_val_score(dt,x,y,cv=10)
score_dt

array([0.52631579, 0.53289474, 0.58552632, 0.64473684, 0.86842105,
       0.90131579, 0.84210526, 0.86184211, 0.86754967, 0.84768212])

In [20]:
score_dt.mean()

0.7478389682816313

In [21]:
# KNN and its cross validation
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
knn.fit(x_train,y_train)
knn_pred=knn.predict(x_test)
accuracy_score(y_test,knn_pred)

0.7521929824561403

In [22]:
score_knn=cross_val_score(knn,x,y,cv=10)
score_knn

array([0.53289474, 0.53947368, 0.57894737, 0.58552632, 0.84868421,
       0.88815789, 0.88157895, 0.91447368, 0.84768212, 0.84768212])

In [23]:
score_knn.mean()

0.7465101080515859

In [24]:
score_knn.std()

0.1548932565798328

In [25]:
table=[[score_lr.mean(),score_dt.mean(),score_knn.mean()]]
table.append([score_lr.std(),score_dt.std(),score_knn.std()])
table
table1=pd.DataFrame(table,index=['Accury_Mean','Accury_SD'])
table1.columns=['Logis Reg','Decision Tree','KNN']
table1

,Logis Reg,Decision Tree,KNN
Accury_Mean,0.748488,0.747839,0.746510
Accury_SD,0.123723,0.147148,0.154893


In [26]:
# implementation stratified k fold for logistic regression
from sklearn.model_selection import StratifiedKFold

In [27]:
accuracy=[]
skf=StratifiedKFold(n_splits=10,random_state=50)
skf.get_n_splits(x,y)  

10

In [28]:
for train_index,test_index in skf.split(x,y):
    x1_train,x1_test=x.iloc[train_index],x.iloc[test_index]
    y1_train,y1_test=y.iloc[train_index],y.iloc[test_index]
    lr.fit(x1_train,y1_train)
    lr1_pred=lr.predict(x1_test)
    sc=accuracy_score(y1_test,lr1_pred)
    accuracy.append(sc)

In [29]:
accuracy

[0.5394736842105263,
 0.5986842105263158,
 0.6052631578947368,
 0.6644736842105263,
 0.8223684210526315,
 0.8618421052631579,
 0.8552631578947368,
 0.8355263157894737,
 0.8278145695364238,
 0.8741721854304636]

In [30]:
np.mean(accuracy)


0.7484881491808993

In [31]:
np.std(accuracy)

0.12372275936731354

In [32]:
# implementation of decision tree through stratified k fold
accuracy1=[]
for train_index,test_index in skf.split(x,y):
    x1_train,x1_test=x.iloc[train_index],x.iloc[test_index]
    y1_train,y1_test=y.iloc[train_index],y.iloc[test_index]
    dt.fit(x1_train,y1_train)
    dt1_pred=dt.predict(x1_test)
    sc1=accuracy_score(y1_test,dt1_pred)
    accuracy1.append(sc1)

In [33]:
accuracy1

[0.4868421052631579,
 0.5263157894736842,
 0.5723684210526315,
 0.6447368421052632,
 0.8618421052631579,
 0.8947368421052632,
 0.8618421052631579,
 0.881578947368421,
 0.8543046357615894,
 0.8543046357615894]

In [34]:
np.mean(accuracy1)

0.7438872429417914

In [35]:
# implementation of KNN through stratified k fold
accuracy2=[]
for train_index,test_index in skf.split(x,y):
    x1_train,x1_test=x.iloc[train_index],x.iloc[test_index]
    y1_train,y1_test=y.iloc[train_index],y.iloc[test_index]
    knn.fit(x1_train,y1_train)
    knn1_pred=knn.predict(x1_test)
    sc2=accuracy_score(y1_test,knn1_pred)
    accuracy2.append(sc2)

In [36]:
accuracy2

[0.5328947368421053,
 0.5394736842105263,
 0.5789473684210527,
 0.5855263157894737,
 0.8486842105263158,
 0.8881578947368421,
 0.881578947368421,
 0.9144736842105263,
 0.847682119205298,
 0.847682119205298]

In [37]:
np.std(accuracy2)

0.1548932565798328

In [38]:
table=[[score_lr.mean(),score_dt.mean(),score_knn.mean()]]
table.append([score_lr.std(),score_dt.std(),score_knn.std()])
table.append([np.mean(accuracy),np.mean(accuracy1),np.mean(accuracy2)])
table.append([np.std(accuracy),np.std(accuracy1),np.std(accuracy2)])
table
table1=pd.DataFrame(table,index=['K-Fold Accury_Mean','K-Fold Accury_SD','StratifiedKFold Acc_Mean','StratifiedKFold Acc_SD'])
table1.columns=['Logis Reg','Decision Tree','KNN']
table1

,Logis Reg,Decision Tree,KNN
K-Fold Accury_Mean,0.748488,0.747839,0.746510
K-Fold Accury_SD,0.123723,0.147148,0.154893
StratifiedKFold Acc_Mean,0.748488,0.743887,0.746510
StratifiedKFold Acc_SD,0.123723,0.157030,0.154893
